# 🔍 N-gram 기반 양손잡이 리더십 키워드 추출

In [1]:
# 라이브러리
from kiwipiepy import Kiwi
import pandas as pd
import re
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv('data/skhynix/leadership_news.csv')
texts = df['content'].dropna().tolist()


In [3]:
# Kiwi 형태소 분석기
kiwi = Kiwi()

# Kiwi 형태소 분석 + 문서 단위 명사/외래어 토큰화 → 문장으로 재조합
def preprocess_for_ngram(texts):
    result_docs = []
    for doc in texts:
        doc = re.sub(r'[^\w\s]', ' ', doc)
        tokens = [token.form for token in kiwi.tokenize(doc) if token.tag.startswith("NN") or token.tag == "SL"]
        result_docs.append(" ".join(tokens))  # 공백으로 연결 → N-gram 적용을 위해
    return result_docs

# 텍스트 전처리
processed_docs = preprocess_for_ngram(texts)

In [4]:
# CountVectorizer로 N-gram 생성 및 빈도 분석 (2~4gram)
vectorizer = CountVectorizer(ngram_range=(2, 4), min_df=2)
X = vectorizer.fit_transform(processed_docs)
ngrams = vectorizer.get_feature_names_out()
counts = X.toarray().sum(axis=0)

# 결과 정리
df_ngrams = pd.DataFrame({
    'ngram': ngrams,
    'count': counts
}).sort_values(by='count', ascending=False)

# 상위 N-gram 출력
df_ngrams.head(50)

,ngram,count
98331,인공 지능,1044
67320,사회 가치,876
51168,메모리 반도체,805
45579,데이터 센터,684
53531,문제 해결,680
35202,기술 개발,624
56233,반도체 산업,564
11160,sk하이닉스 구성원,533
17745,가치 창출,439
109664,제품 개발,414


## 사전 기반 키워드 매칭 (Exact Match / 포함 여부 기반)

In [5]:
# 사전 키워드 (공백 기준)
exploration_dict  = [
    "신사업", "플랫폼 전략", "디지털 전환", "AI 반도체", "차세대 공정", "3D DRAM", "HBM",
    "미래 기술", "오픈이노베이션", "R&D 투자 확대", "기술독립", "기술 자립화", "첨단소재 개발",
    "유연한 조직", "벤처 협업", "융합 기술", "자율성 부여", "위험 감수", "장기 전략",
    "파일럿 프로젝트", "혁신 생태계", "아이디어 제안", "기술 실험", "고객 중심 혁신",
    "차세대 패키징", "설계자산 확보", "경쟁력 있는 기술 선도", "초격차 전략", "선도 기술 확보",
    "창의적 조직 문화"
]
exploitation_dict  = [
    "공정 최적화", "양산 체계", "수율 향상", "공정 자동화", "생산성 향상", "설비 효율화",
    "원가 절감", "재고 관리", "운영 안정성", "ROI 극대화", "공급망 관리", "품질 인증",
    "생산 리드타임 단축", "에너지 효율 개선", "리스크 관리 체계", "수익성 강화", "정해진 목표 달성",
    "표준 공정 정립", "성과 관리 시스템", "ESG 대응 체계화", "클린룸 관리", "ERP 기반 통제",
    "운영 KPI 관리", "작업표준 확립", "공정 불량률 관리", "프로세스 개선", "업무 매뉴얼화",
    "규정 준수 강화", "직무 책임 강화", "조직 내 수직 협업"
]

# 탐색/활용 라벨링 함수
def classify_ngram(ngram):
    for key in exploration_dict:
        if key in ngram:
            return "탐색"
    for key in exploitation_dict:
        if key in ngram:
            return "활용"
    return "기타"

# 적용
df_ngrams['category'] = df_ngrams['ngram'].apply(classify_ngram)

# 결과 확인
df_ngrams[df_ngrams['category'] != "기타"].sort_values(by='count', ascending=False).head(30)


,ngram,count,category
54542,미래 기술,343,탐색
54564,미래 기술 연구원,174,탐색
92971,원가 절감,54,활용
83961,아이디어 제안,52,탐색
11735,sk하이닉스 미래 기술,50,탐색
48297,디지털 전환,41,탐색
77725,수율 향상,38,활용
11737,sk하이닉스 미래 기술 연구원,36,탐색
54575,미래 기술 연구원 담당,32,탐색
94709,융합 기술,20,탐색


## 유사도 기반 키워드 매칭 (변형된 표현 처리)

In [8]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def classify_fuzzy(ngram):
    for kw in exploration_dict:
        if similar(kw, ngram) > 0.8:
            return "탐색"
    for kw in exploitation_dict:
        if similar(kw, ngram) > 0.8:
            return "활용"
    return "기타"

# 적용
df_ngrams['category_fuzzy'] = df_ngrams['ngram'].apply(classify_fuzzy)
# 결과 확인
df_ngrams[df_ngrams['category_fuzzy'] != "기타"].sort_values(by='count', ascending=False).head(30)



,ngram,count,category,category_fuzzy
54542,미래 기술,343,탐색,탐색
70274,생산 향상,63,기타,활용
75976,소재 개발,60,기타,탐색
92971,원가 절감,54,활용,활용
83961,아이디어 제안,52,탐색,탐색
48297,디지털 전환,41,탐색,탐색
77725,수율 향상,38,활용,활용
26765,공정 최적,33,기타,활용
94709,융합 기술,20,탐색,탐색
48261,디지털 변환,13,기타,탐색
